In [30]:
#The First step is to import useful dependencies

#It is useful in creating numpy arrays
import numpy as np
#It is useful for creating and storing data into data_frames
import pandas as pd
#It is useful for searching a comment or a word in a text or paragraph
import re

#In this we are using  NLPtoolkit model and importing its "stopwards" module and this stopwards help us to not include those 
#words that do not add value to our paragraph example:- the ,this that, and, it etc.
from nltk.corpus import stopwords
#this funtion is used to stem our words and hence find the root word.
from nltk.stem.porter import PorterStemmer
#In this we are importing our text into Vectors with the help of imported module tfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#This help us to Split our data into text data and train data
from sklearn.model_selection import train_test_split
#In this we are importing the "Logistic_Regression_Model" which will help in Classification
from sklearn.linear_model import LogisticRegression
#
from sklearn.metrics import accuracy_score

In [44]:
#In this we will bw importing nltk that is natural language toolkit that we will use later.
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jbbon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# printing all the stopwards in English language
print(stopwords.words("English"))

# print(stopwords.words('French'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Now The DATA PRE_PROCESSING Part Starts


In [46]:
#now lets load the dataset into a dataframe
news_dataset=pd.read_csv("train.csv")

In [47]:
news_dataset.head()
# label 1 :- Represents Fake News
# label 2 :- Represents real News

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
news_dataset.shape

(20800, 5)

In [49]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [50]:
#As There Are few missing values so let's Put Null String On place of Empty values
news_dataset=news_dataset.fillna('')

In [51]:
#let See now how may values are empty Now
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [52]:
#We Will Check our model on Author and title only as the text is very large so traning it may consume your most of the time 
#so to do that we will merge both The Title Column and Author Column

news_dataset['content']=news_dataset['author']+news_dataset['title']
news_dataset['content'].head()

0    Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1    Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2    Consortiumnews.comWhy the Truth Might Get You ...
3    Jessica Purkiss15 Civilians Killed In Single U...
4    Howard PortnoyIranian woman jailed for fiction...
Name: content, dtype: object

In [53]:
#Removing the label from dataset and storing it in A varible name "X"
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']
#let's See these X and Y
print(X)
# print("\n")
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# Stemming Is the Process of Reducing word to its Root word
#For Example:- Actress, Actor, Acting --> act

In [62]:
#
port_stem=PorterStemmer()

In [63]:
#Now We Are creating a function that will remove all the non-beneficial values 
def stemming(content):
    #in this step we are using re module to search only alphabets and remove every other things like , ,"",numbers etc.
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    #now convert all the letters into lowercase letters
    stemmed_content = stemmed_content.lower()
    #now all the text is splitted and converted into list
    stemmed_content = stemmed_content.split()
    #in this step we are running a for loop and storing all the non stopword present in "stemmed_content".
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [64]:
#Now Let's Apply this stemming on our "news_dataset['content']"
news_dataset['content']=news_dataset['content'].apply(stemming)
news_dataset['content'].head()

0    darrel lucush dem aid even see comey letter ja...
1    daniel j flynnflynn hillari clinton big woman ...
2            consortiumnew comwhi truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoyiranian woman jail fiction unpub...
Name: content, dtype: object

In [65]:
#In this step we are seprating data and values
X=news_dataset['content'].values
Y=news_dataset['label'].values
print(X)

['darrel lucush dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exerci balkan'
 'david swansonwhat keep f aliv']


In [66]:
#Converting Textual Data To Numerical Data
#This Tf stands for term frequency :-  this basically counts how many times a particular word is repeated in paragraph and
# hence assign it a larger value because it may be important that's why it is repested many times.
# and this idf stands for inverse difi frequency :- this states that if a word is repeated many times doesn't mean it will 
# have more importance or meaning in it
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
# print(X)

In [67]:
print(X)

  (0, 26171)	0.28088379401596425
  (0, 22568)	0.2552336018069161
  (0, 14920)	0.43006226759639316
  (0, 14460)	0.29177259684200296
  (0, 12695)	0.24619727512767195
  (0, 7961)	0.23133661742488731
  (0, 6233)	0.2839932825877813
  (0, 5930)	0.35488202138141456
  (0, 4975)	0.2472595823572816
  (0, 4187)	0.3625320323150658
  (0, 572)	0.2694167078545385
  (1, 27744)	0.37204408227449065
  (1, 11235)	0.2434639969074374
  (1, 8704)	0.5300315426405939
  (1, 5877)	0.3206218375341294
  (1, 4739)	0.23523739119536424
  (1, 3838)	0.44601381123255807
  (1, 3260)	0.18800278047529806
  (1, 2602)	0.35911932371235855
  (2, 26066)	0.3665032495181434
  (2, 16256)	0.43295215406038445
  (2, 9384)	0.30743020569262086
  (2, 8499)	0.3411947414020896
  (2, 5204)	0.40440534260277944
  (2, 5089)	0.5511414848555652
  :	:
  (20797, 25609)	0.08234347981192172
  (20797, 25155)	0.3125002447493035
  (20797, 21934)	0.2494515865666979
  (20797, 20637)	0.2734270447609076
  (20797, 20353)	0.25042469546773877
  (20797, 17396

# Spliting Out Data Into Train And Test Data

In [75]:
#In this we are splitint data into four parts that is x train xtest, ytrain and ytest in 80-20 ratio
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=3)

In [76]:
#Traing The Model(LOGISTIC_REGRESSION)
model=LogisticRegression()

In [77]:
model.fit(X_train,Y_train)

LogisticRegression()

In [78]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [79]:
print("The Accuracy score is :- ",training_data_accuracy)

The Accuracy score is :-  0.97109375


In [81]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [82]:
print("The Accuracy score is :- ",test_data_accuracy)

The Accuracy score is :-  0.9557692307692308


In [90]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real
